In [1]:
def now():
    print('2018-03-08')

f = now
f()
#函数对象有一个__name__属性，可以拿到函数的名字
print("now.__name__=",now.__name__)
print("f.__name__=",f.__name__)


2018-03-08
now.__name__= now
f.__name__= now


In [5]:
#现在，假设我们要增强now()函数的功能，比如，在函数调用前后自动打印日志，但又不希望修改now()函数的定义，这种在代码运行期间动态增加功能的方式，称之为“装饰器”（Decorator）。
#本质上，decorator就是一个返回函数的高阶函数。所以，我们要定义一个能打印日志的decorator，可以定义如下：
def log(func):
    def wrapper(*args, **kw):
        print('call %s():' %func.__name__)
        return func(*args, **kw)
    return wrapper
#把@log放到now()函数的定义处，相当于执行了语句：now = log(now)
@log
def now():
    print('2018-03-08')
    
now()
#由于log()是一个decorator，返回一个函数，所以，原来的now()函数仍然存在，只是现在同名的now变量指向了新的函数，于是调用now()将执行新函数，即在log()函数中返回的wrapper()函数。
#wrapper()函数的参数定义是(*args, **kw)，因此，wrapper()函数可以接受任意参数的调用。在wrapper()函数内，首先打印日志，再紧接着调用原始函数。

call now():
2018-03-08


In [7]:
#如果decorator本身需要传入参数，那就需要编写一个返回decorator的高阶函数，写出来会更复杂。比如，要自定义log的文本：
def log(text):
    def decorator(func): 
        def wrapper(*args, **kw):
            print('%s %s():' %(text,func.__name__))
            return func(*args, **kw)
        return wrapper
    return decorator
#此处相当于执行了语句：now = log('execute')(now)
#首先执行log('execute')，返回的是decorator函数，再调用返回的函数，参数是now函数，返回值最终是wrapper函数。
@log('execute')
def now():
    print('2018-03-08')
    
now()

#函数也是对象，有__name__等属性。经过decorator装饰之后的函数，它们的__name__已经从原来的'now'变成了'wrapper'：
print(now.__name__)
#因为返回的那个wrapper()函数名字就是'wrapper'，所以，需要把原始函数的__name__等属性复制到wrapper()函数中，否则，有些依赖函数签名的代码执行就会出错。

execute now():
2018-03-08
wrapper


In [9]:
#不需要编写wrapper.__name__ = func.__name__这样的代码，Python内置的functools.wraps就是干这个事的，所以，一个完整的decorator的写法如下：
import functools

def log(func):
    @functools.wraps(func)
    def wrapper(*args,**kw):
        print('call %s():' % func.__name__)
        return func(*args,**kw)
    return wrapper

@log
def now():
    print('2018-03-08')
    
now()
print(now.__name__)

call now():
2018-03-08
now


In [10]:
#针对带参数的decorator：
import functools

def log(text):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kw):
            print('%s %s():' % (text, func.__name__))
            return func(*args,**kw)
        return wrapper
    return decorator

@log('execute')
def now():
    print('2018-03-08')

now()
print(now.__name__)

execute now():
2018-03-08
now


In [2]:
#练习：设计一个decorator，它可作用于任何函数上，并打印该函数的执行时间：
import functools
import time

def metric(fn):
    @functools.wraps(fn)
    def wrapper(*args, **kw):
        begin = time.time()
        result = fn(*args, **kw)
        print('%s executed in %.4f ms' % (fn.__name__, time.time()-begin))
        return result
    return wrapper

# 测试
@metric
def fast(x, y):
    time.sleep(0.0012)
    return x + y;

@metric
def slow(x, y, z):
    time.sleep(0.1234)
    return x * y * z;

f = fast(11, 22)
s = slow(11, 22, 33)
if f != 33:
    print('测试失败!')
elif s != 7986:
    print('测试失败!')
    
#思考：设计一个装饰器，既可带参，也可不带参。即：
# @log
# def f():
#     pass

# @log('execute')
# def f():
#     pass


fast executed in 0.0156 ms
slow executed in 0.1248 ms


In [ ]:
#其他参考：http://blog.csdn.net/u013471155/article/details/68960244